# Inference Validation Notebook

TL;DR - 

This notebook is a general purposes that serves to run [trained models](https://huggingface.co/kamel-usp) so we can validate reported results

In [1]:
from torch.utils.data import DataLoader
from datasets import load_dataset, DatasetDict, Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import accuracy_score, cohen_kappa_score, mean_squared_error
import pytorch_lightning as pl
import torch
from tqdm.auto import tqdm
import pandas as pd
from coral_pytorch.dataset import corn_label_from_logits


RANDOM_SEED = 42
REFERENCE_CONCEPT = 0
OBJECTIVE = "ordinal"
MAX_LENGTH = 512
BATCH_SIZE=16
VARIANT = "base" #base/large
TOKENIZER_NAME = f"neuralmind/bert-{VARIANT}-portuguese-cased"
BASE_MODEL = "bertimbau" #bertimbau/sourceB-mlm/sourceB-ordinal
MODEL_NAME = f"kamel-usp/aes_enem_models-sourceA-ordinal-from-{BASE_MODEL}-{VARIANT}-C{REFERENCE_CONCEPT+1}"

pl.seed_everything(RANDOM_SEED)
torch.set_float32_matmul_precision('medium')
MODEL_NAME

Seed set to 42


'kamel-usp/aes_enem_models-sourceA-ordinal-from-bertimbau-base-C1'

In [2]:
dataset = load_dataset("kamel-usp/aes_enem_dataset", "sourceAWithGraders", cache_dir="/tmp/aes_enem")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'id_prompt', 'essay_title', 'essay_text', 'grades', 'essay_year'],
        num_rows: 744
    })
    validation: Dataset({
        features: ['id', 'id_prompt', 'essay_title', 'essay_text', 'grades', 'essay_year'],
        num_rows: 195
    })
    test: Dataset({
        features: ['id', 'id_prompt', 'essay_title', 'essay_text', 'grades', 'essay_year'],
        num_rows: 216
    })
})

In [4]:
grade_mapping = {
    0: 0,
    40: 1,
    80: 2,
    120: 3,
    160: 4,
    200: 5,
}

def create_label(row):
    grade = row["grades"][REFERENCE_CONCEPT]
    return {"label": grade_mapping[grade]}

dataset = dataset.map(create_label)

In [5]:
def compute_difference(lists):
    # Assuming the first element is the reference for subtraction
    reference = lists[0][REFERENCE_CONCEPT]
    grader_a = lists[1][REFERENCE_CONCEPT]
    grader_b = lists[2][REFERENCE_CONCEPT]

    # Calculate absolute differences
    diff_ref_a = abs(reference - grader_a)
    diff_ref_b = abs(reference - grader_b)
    diff_a_b = abs(grader_a - grader_b)

    # Check if any difference is greater than 80
    return diff_ref_a > 80 or diff_ref_b > 80 or diff_a_b > 80

test_df = dataset["test"].to_pandas()
new_test_df = pd.merge(
    test_df.groupby(["id_prompt", "id"]).agg({"grades": list}).apply(lambda x: compute_difference(x['grades']), axis=1).reset_index(),
    test_df,
    on=["id_prompt","id"]
).rename(columns={0: "is_hard"})

In [6]:
dataset["test_easy"] = Dataset.from_pandas(new_test_df[new_test_df["is_hard"]==False])
dataset["test_hard"] = Dataset.from_pandas(new_test_df[new_test_df["is_hard"]==True])

In [7]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME, use_fast=True)
def get_model_instance(model_path, objective):
    model = None
    if objective == "regression":
        model = AutoModelForSequenceClassification.from_pretrained(
                model_path, 
                cache_dir="/tmp/", 
                num_labels=1,
            )
    elif objective == "classification" or objective == "ordinal":
        model = AutoModelForSequenceClassification.from_pretrained(
                model_path, 
                cache_dir="/tmp/", 
                num_labels=6,
            )
    return model
model = get_model_instance(MODEL_NAME, OBJECTIVE)
if model is None:
    raise ValueError("Please set a Pre defined Objective")

In [8]:
def prepare_dataset(dataset):
    def tokenize_essays(dataset, tokenizer, max_length=512):
        tokenized_text = tokenizer(
                dataset["essay_text"],
                return_tensors="pt",
                truncation=True,
                padding="max_length",
                max_length=max_length
            )
        tokenized_text["label"] = dataset["label"]
        return tokenized_text
    
    tokenized_datasets = {
        split: tokenize_essays(sub_dataset, tokenizer, MAX_LENGTH)
        for split, sub_dataset in dataset.items()
    }
    dataset_tokenized = DatasetDict({
        split: Dataset.from_dict(data)
        for split, data in tokenized_datasets.items()
    })

    return dataset_tokenized

dataset_tokenized = prepare_dataset(dataset)

In [9]:
data_train = DataLoader(
    dataset_tokenized["train"].with_format("torch"), batch_size=BATCH_SIZE, shuffle=True, num_workers=0
)
data_val = DataLoader(dataset_tokenized["validation"].with_format("torch"), batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
data_test = DataLoader(dataset_tokenized["test"].with_format("torch"), batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

data_test_easy = DataLoader(dataset_tokenized["test_easy"].with_format("torch"), batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
data_test_hard = DataLoader(dataset_tokenized["test_hard"].with_format("torch"), batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [10]:
def predict_classes(output):
    if OBJECTIVE == "regression":
        # Round the tensor to the nearest integer
        rounded_tensor = torch.round(output.logits)
        # Clamp the values to the range [0, 5]
        clamped_tensor = torch.clamp(rounded_tensor, min=0, max=5)
        return clamped_tensor.view(-1)
    elif OBJECTIVE == "classification":
        return torch.argmax(output.logits, axis=1)
    elif OBJECTIVE == "ordinal":
        return corn_label_from_logits(output.logits)
        
def get_predictions_and_labels(model, dataloader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    all_predictions = []
    all_true_labels = []
    i=0
    for batch in tqdm(dataloader, desc="Obtaining predictions"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)
        with torch.no_grad():
            output = model(input_ids, attention_mask)
            predicted_classes = predict_classes(output) 

        # If using GPU, need to move the data back to CPU to use numpy.
        all_predictions.extend(predicted_classes.cpu().numpy())
        all_true_labels.extend(labels.cpu().numpy())

    return list(map(lambda x: x * 40, all_predictions)), list(map(lambda x: x * 40, all_true_labels))

In [11]:
def enem_accuracy_score(true_values, predicted_values):
    assert len(true_values) == len(predicted_values), "Mismatched length between true and predicted values."

    non_divergent_count = sum([1 for t, p in zip(true_values, predicted_values) if abs(t - p) <= 80])
    
    return non_divergent_count / len(true_values)

In [12]:
all_predictions, all_true_labels = get_predictions_and_labels(model, data_test)
accuracy = accuracy_score(all_true_labels, all_predictions)
print(f"Accuracy on the validation  set: {accuracy:.2f}")

Obtaining predictions:   0%|          | 0/14 [00:00<?, ?it/s]

Accuracy on the validation  set: 0.49


In [13]:
qwk = cohen_kappa_score(all_true_labels, all_predictions, weights="quadratic", labels=[0,40,80,120,160,200])
print(f"QWK on the validation set: {qwk:.2f}")

QWK on the validation set: 0.30


In [14]:
enem_accuracy = enem_accuracy_score(all_true_labels, all_predictions)
print(f"Accuracy on the validation set: {enem_accuracy:.2f}")

Accuracy on the validation set: 0.93


In [15]:
def compute_metrics(model, dataset, test_group):
    all_predictions, all_true_labels = get_predictions_and_labels(model, dataset)
    accuracy = accuracy_score(all_true_labels, all_predictions)
    qwk = cohen_kappa_score(all_true_labels, all_predictions, weights="quadratic", labels=[0,40,80,120,160,200]) 
    rmse = mean_squared_error(all_true_labels, all_predictions, squared=False)
    horizontal_discrepancy = enem_accuracy_score(all_true_labels, all_predictions)
    result = {
        'Experiment Reference': MODEL_NAME,
        'Test Group': test_group,
        'Competence': REFERENCE_CONCEPT,
        'Accuracy': [accuracy],
        'RMSE': [rmse],
        'QWK': [qwk],
        'HDIV': [1- horizontal_discrepancy]
    }
    return pd.DataFrame(result)

In [19]:
model.eval()
display(compute_metrics(model, data_test, "full"))
display(compute_metrics(model, data_test_easy, "easy"))

Obtaining predictions:   0%|          | 0/14 [00:00<?, ?it/s]

,Experiment Reference,Test Group,Competence,Accuracy,RMSE,QWK,HDIV
0,kamel-usp/aes_enem_models-sourceA-ordinal-from...,full,0,0.486111,44.472214,0.295411,0.074074


Obtaining predictions:   0%|          | 0/11 [00:00<?, ?it/s]

,Experiment Reference,Test Group,Competence,Accuracy,RMSE,QWK,HDIV
0,kamel-usp/aes_enem_models-sourceA-ordinal-from...,easy,0,0.541667,31.622777,0.427347,0.011905


In [17]:
MODEL_NAME

'kamel-usp/aes_enem_models-sourceA-ordinal-from-bertimbau-base-C1'